# 1. The basics - making an API request

In [ ]:
!pip3 install --upgrade pip

In [ ]:
!pip3 install openai python-dotenv streamlit

In [ ]:
# import required packages

# FILL IN
import json
from dotenv import load_dotenv
from pprint import pprint
import re

load_dotenv()

In [ ]:
# FILL IN

### Basic chat completion

In [ ]:
completion # FILL
  model= # FILL IN
  messages=[
    {"role": "system", "content": "You are an expert fitness trainer that specializes in helping people prep for the first competitive event."},
    {"role": "user", "content": "Provide me with a training plan to prepare for my 5k in 4 weeks. I want to run it in under 28 minutes."}
  ],
  temperature= # FILL IN
  max_tokens= # FILL IN
  seed= # FILL IN
  
)
print(...)

In [ ]:
# view response
# FILL IN

### JSON Mode

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are an expert fitness trainer that specializes in helping people prep for the first competitive event. \
      You respond in a JSON format, where the keys represent weeks and days, and the values contain the training plan."},
    {"role": "user", "content": "Provide me with a training plan to prepare for my 5k in 4 weeks. I want to run it in under 28 minutes."}
  ],
  temperature=1,
  max_tokens=1024,
  seed=42,
  # FILL IN
  
)

print(completion.choices[0].message.content)


### Streaming Mode

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are an expert fitness trainer that specializes in helping people prep for the first competitive event."},
    {"role": "user", "content": "Provide me with a training plan to prepare for my 5k in 4 weeks. I want to run it in under 28 minutes."}
  ],
  temperature=1,
  max_tokens=1024,
  seed=42,
  # FILL IN
  
)

for chunk in completion:
  print(chunk.choices[0].delta.content)


# 2. Using Tools

Get Current Weather - Dummy Tool

In [ ]:
# define dummy tool function
def get_current_weather(location, unit='fahreinheit'):
    if 'tokyo' in location.lower():
        return json.dumps({'location': 'Tokyo', 'temperature': '10', 'unit': unit})
    elif 'san francisco' in location.lower():
        return json.dumps({'location': 'San Francisco', 'temperature': '72', 'unit': unit})
    elif 'paris' in location.lower():
        return json.dumps({'location': 'Tokyo', 'temperature': '22', 'unit': unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
def run_conversation(user_input):
    messages = # FILL IN
    tools = [
        {
            'type': # FILL IN
            'function': {
                'name': # FILL IN
                'description': # FILL IN
                'parameters': {
                    'type': 'object',
                    'properties': {
                        # FILL IN
                        'unit': {'type': 'string', 'enum': ['celsius', 'fahreinheit']}

                        },
                        'required': # FILL IN
                    }
                }
            }
    ]
    
    # call model and give it tools
    response = client.chat.completions.create(
        model= # FILL IN
        messages=messages,
        tools= # FILL IN
        tool_choice= # FILL IN
    )

    # print out model response
    response_message = response.choices[0].message
    print('Tool Call Response:')
    # FILL IN
    print()

    # print out if model chose to call a tool
    tool_calls = response_message.tool_calls
    print('Tool Calls:')
    # FILL IN
    print()
    
    # if a tool was called, extract tool arguments and execute tool
    if # FILL IN
        available_functions = {
            "get_current_weather": get_current_weather,
        }
        messages.append(response_message)
        for # FILL IN
            function_name = # FILL IN
            function_to_call = available_functions[function_name]
            function_args = # FILL IN
            function_response = function_to_call(
                location= # FILL
                unit=function_args.get("unit")
            )        
            # append tool response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": # FILL IN
            })
        # call model again with updated messages
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages= # FILL IN
        )
        return second_response.choices[0].message.content

In [ ]:
# Test function calling
user_input = # FILL IN
response = # FILL IN

print(("Tool Output:"))
print(f'User: {user_input}')
print(f'Assistant: {response}')

# 3. Key prompting techniques

In [ ]:
cot_fsp_system_prompt = """You are a health and fitness coach specializing in preparing people for athletic events like 5K runs, mud runs, and triathlons. 
Think step by step to create a personalized training plan. 
Produce your thinking output in <thinking> tags and your client-facing message output in <message> tags. 
Ask a few questions to understand the client's needs and then suggest a training plan. 
Answer simply and avoid overly narrating. Only ask 1 question at a time

When talking to users, follow these examples:

Example 1:
User: I want to run a 5K in 8 weeks. How should I train?

Assistant: <thinking> 
I need to:
1. Determine user's current fitness level.
2. Assess how many days per week the user can train.
3. Create a progressive training plan that builds up to 5K over 8 weeks.
I will start by asking about their current fitness level.
</thinking>
<message> 
To create a tailored training plan, I need to know:
1. What is your current fitness level? (e.g., beginner, intermediate, advanced)
</message>

User: I'm a beginner
Assistant: 
<thinking> 
I need to: 
1. Note the user's beginner status.
2. Assess how many days per week the user can train.
2. Create a progressive training plan that builds up to 5K over 8 weeks.
</thinking>
<message> 
Great! How many days per week can you commit to training?
</message>

User: I can train 3 days a week.
Assistant: <thinking> 
I need to: 
1. Note the user's beginner status.
2. Create a training plan that builds up to 5K with 3 days of training per week.
3. Ensure the plan includes rest and recovery periods.
</thinking>
<message> 
Great! Here's a basic 8-week plan for a beginner training 3 days a week:
- Weeks 1-2 : Walk/jog intervals for 20-30 minutes.
- Weeks 3-4: Jog for 20-30 minutes.
- Weeks 5-6: Jog for 30-40 minutes.
- Weeks 7-8: Run for 30-40 minutes.
Make sure to include rest days between training sessions.
</message>
"""

In [ ]:
# get response from model chat completion
def get_response(messages):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    seed=42
    
  )
  return completion.choices[0].message.content

In [ ]:
# parse response to extract thinking and message content
def parse_response(response):
    thinking_pattern = # FILL IN
    message_pattern = r"<message>(.*?)</message>"

    thinking = re.findall("*** FILL IN ***", re.DOTALL)
    message = re.findall(message_pattern, response, re.DOTALL)

    return (thinking[0] if thinking else "", message[0] if message else "")

In [ ]:
messages=[
    {"role": "system", "content": # FILL IN
  ]

while True:
  user_input = input("Enter your message...")
  if user_input.lower() == "quit":
    break
  messages.append({"role": "user", "content": # FILL IN
  print(f'User: {user_input}')

  response = # FILL IN
  messages.append({"role": "assistant", "content": response})

  thinking, message = # FILL IN
  print(f'Assistant thinking (internal): {thinking}')
  print(f'Assistant message (external): {message}')